In [6]:
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
#data collection - kaggle API
kaggle_dictionary = json.load(open('kaggle.json'))

In [3]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [4]:
#set upkaggle API credentials as environment variables
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [16]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other




  0%|          | 0.00/25.7M [00:00<?, ?B/s]
  4%|▍         | 1.00M/25.7M [00:02<00:55, 466kB/s]
  8%|▊         | 2.00M/25.7M [00:06<01:21, 306kB/s]
 12%|█▏        | 3.00M/25.7M [00:16<02:28, 161kB/s]
 16%|█▌        | 4.00M/25.7M [00:23<02:25, 156kB/s]
 19%|█▉        | 5.00M/25.7M [00:29<02:13, 163kB/s]
 23%|██▎       | 6.00M/25.7M [00:37<02:18, 149kB/s]
 27%|██▋       | 7.00M/25.7M [00:46<02:18, 142kB/s]
 31%|███       | 8.00M/25.7M [00:52<02:05, 149kB/s]
 35%|███▌      | 9.00M/25.7M [00:57<01:47, 162kB/s]
 39%|███▉      | 10.0M/25.7M [01:05<01:50, 149kB/s]
 43%|████▎     | 11.0M/25.7M [01:18<02:10, 118kB/s]
 47%|████▋     | 12.0M/25.7M [01:26<01:55, 125kB/s]
 51%|█████     | 13.0M/25.7M [01:33<01:44, 128kB/s]
 54%|█████▍    | 14.0M/25.7M [01:42<01:35, 128kB/s]
 58%|█████▊    | 15.0M/25.7M [01:48<01:21, 139kB/s]
 62%|██████▏   | 16.0M/25.7M [01:54<01:10, 145kB/s]
 66%|██████▌   | 17.0M/25.7M [02:01<01:00, 150kB/s]
 70%|███████   | 18.0M/25.7M [02:12<01:03, 128kB/s]
 74%|███████▍  | 19

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [23]:
os.listdir('imdb_data')

['IMDB Dataset.csv']

In [18]:
#unzip the dataset
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
    zip_ref.extractall('imdb_data')

In [20]:
os.listdir('imdb_data')

['IMDB Dataset.csv']

In [25]:
#loading the dataset
data = pd.read_csv('IMDB Dataset.csv')

In [26]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [27]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [28]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [29]:
data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)

C:\Users\badri\AppData\Local\Temp\ipykernel_24848\453906489.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)


In [30]:
data['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [31]:
#splitting data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)  

In [32]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [38]:
#tokenising the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
#padding the sequences - makes all sequences of equal length
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [39]:
print(X_train.shape)
print(X_test.shape)

(40000, 200)
(10000, 200)


In [41]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [42]:
print(Y_train)
print(Y_test)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64
33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


In [45]:
#Long Short-Term Memory (LSTM) Model - used for sequence prediction problems
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [48]:
model.build(input_shape=(None, 200))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
#compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [53]:
#training the model
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 159s 319ms/step - accuracy: 0.8437 - loss: 0.3689 - val_accuracy: 0.8564 - val_loss: 0.3511
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 147s 294ms/step - accuracy: 0.8776 - loss: 0.3024 - val_accuracy: 0.8806 - val_loss: 0.2892
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 161s 321ms/step - accuracy: 0.8584 - loss: 0.3336 - val_accuracy: 0.8605 - val_loss: 0.3465
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 90s 179ms/step - accuracy: 0.8983 - loss: 0.2596 - val_accuracy: 0.8788 - val_loss: 0.3032
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 97s 194ms/step - accuracy: 0.9176 - loss: 0.2131 - val_accuracy: 0.8792 - val_loss: 0.3060


In [52]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 56ms/step - accuracy: 0.8551 - loss: 0.3447
Test Loss: 0.3446885049343109
Test Accuracy: 0.8550999760627747


In [51]:
#building a prediction function
def predict_sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = 'positive' if prediction >= 0.5 else 'negative'
    return sentiment

In [54]:
#example usage
review_text = "The movie was fantastic! I really enjoyed it."
predicted_sentiment = predict_sentiment(review_text)
print(f'Review: {review_text}')
print(f'Predicted Sentiment: {predicted_sentiment}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step
Review: The movie was fantastic! I really enjoyed it.
Predicted Sentiment: positive
